In [ ]:
import numpy as np
import scipy as sp
from scipy.integrate import quad,cumtrapz
import matplotlib.pyplot as plt


def f_transform(fun,f,lim = None):
    operator_e = lambda w,t:cos(w*t)
    operator_o = lambda w,t:sin(w*t)
    if lim == None:
        even = integrate.quad(lambda t:fun(t)*operator_e(f,t),-np.inf,np.inf)[0]
        odd = integrate.quad(lambda t:fun(t)*operator_o(f,t),-np.inf,np.inf)[0]
    else:
        even = integrate.quad(lambda t:fun(t)*operator_e(f,t),lim[0],lim[1])[0]
        odd = integrate.quad(lambda t:fun(t)*operator_o(f,t),lim[0],lim[1])[0]        
    #return 1/(sqrt(2*pi))*(even - 1j*odd)
    return (even - 1j*odd)


from scipy.fft import fft,fftshift,fftfreq



In [ ]:
x = np.linspace(-1,1,100)
y = lambda t:1 if np.abs(t)<0.5 else 0
plt.plot(x,[y(i) for i in x])
